In [1]:
import sys
import regex
import cgatcore.iotools as iotools
import pysam
import logging
import argparse

In [9]:
with pysam.FastxFile("TrimerTSO_2.fastq.gz") as fh:
    outfile = open("large.fastq", "w")
    n = 0
    for record in fh:
        n += 1
        if len(record.sequence) < 300:
            pass
        else:
            outfile.write("@%s\n%s\n+\n%s\n" % (record.name, record.sequence, record.quality))


In [3]:

tab = str.maketrans("ACTG", "TGAC")

def reverse_complement_table(seq):
    return seq.translate(tab)[::-1]


outfile = open("polyA.fastq", "w")
log =  iotools.open_file("polyA.fastq" + ".log","w")
n = 0
y = 0
with pysam.FastxFile("TrimerTSO_cat.fastq.gz") as fh:
    
    for record in fh:
        y +=1
        if len(record.sequence) < 300:
            pass
        else:
            seq = record.sequence[50:300]
            m=regex.findall("(TTTTTTTTTTTTTTTTTTTT){e<=3}", str(seq))
            if m:
                n +=1
                sequence = reverse_complement_table(str(record.sequence))
                outfile.write("@%s\n%s\n+\n%s\n" % (record.name, sequence, record.quality))
            else:
                seq = record.sequence[-30:-200:]
                m=regex.findall("(AAAAAAAAAAAAAAAAAAAA){e<=3}", str(seq))
                if m:
                    n +=1
                    outfile.write("@%s\n%s\n+\n%s\n" % (record.name, record.sequence, record.quality))
outfile.close()
log.close()

Identify UMI sequences for TSO

In [49]:
tab = str.maketrans("ACTG", "TGAC")

def reverse_complement_table(seq):
    return seq.translate(tab)[::-1]


outfile = open("TSO_UMI.fastq", "w")

n = 0
with pysam.FastxFile("polyA.fastq") as fh:
    
    for record in fh:
        n += 1
        seq_nano = record.sequence
        
        m=regex.finditer("(AAGCAGTGGTATCAACGCAGAGTAAT){e<=3}", str(record.sequence))
        for i in m:
            after_smart = seq_nano[i.end()+1:]
            umi_tso = after_smart[:18]
            after_umi = after_smart[19:]
            record_new = record.name + "_" + str(umi_tso)
            quality_afterumismart = record.quality[i.end()+20:]
            seq_afterumismart = seq_nano[i.end()+20:]
            if len(umi_tso) == 18:
                # at the moment I am not including the umi for tso in the name
                outfile.write("@%s\n%s\n+\n%s\n" % (record.name, seq_afterumismart, quality_afterumismart))
            else:
                print(umi_tso)
            


CCTTCCCAAACCTC
TTTTAAAAAAATTTTTT
CTGCCCGTG
CCAAAAACCCAAAT
CTTAAACCCAAT

TCCCCCTGAAT
T
T
GCAGGGAATAATG
TTGGGAAAAATTT
TCTGGGCAG
CTAAATTTGAAAATTT
ACC
TCCAAT
CCAAAATTTG
GCCCT
GCGGCCCTTTTTCCT
CTGAACGAAGCATACCG
CTTGGGCCCAAACCTG


Identify UMI sequences for polyA

In [53]:
tab = str.maketrans("ACTG", "TGAC")

def reverse_complement_table(seq):
    return seq.translate(tab)[::-1]


outfile = open("polyA_UMI.fastq", "w")

n = 0
with pysam.FastxFile("TSO_UMI.fastq") as fh:
    
    for record in fh:
        n += 1
        seq_nano = record.sequence
        
        m=regex.finditer("(GTACTCTGCGTTGATACCACTGCTT){e<=3}", str(record.sequence))
        
        for i in m:
            after_polya = seq_nano[i.start()-18:]
            umi_polya = seq_nano[i.start()-18:i.start()]
            after_umi = seq_nano[:i.start()-18]
            record_new = record.name + "_" + str(umi_polya)
            quality_afterumipolya = record.quality[:i.start()-36]
            seq_afterumipolya = seq_nano[:i.start()-36]
            
            if len(umi_polya) == 18:
                outfile.write("@%s\n%s\n+\n%s\n" % (record_new, seq_afterumipolya, quality_afterumipolya))
            else:
                pass

            

Map with minimap2 using following options:


minimap2 -ax map-ont -p 0.9 --end-bonus 10 -N 3 Human-Mouse.fa polyA_UMI.fastq

samtools view -bS polyA_UMI.sam > final.bam &&
                   samtools sort final.bam -o final_sorted.bam &&
                   samtools index final_sorted.bam

Then add transcript to XT tag




In [54]:
samfile = pysam.AlignmentFile("final_sorted.bam", "rb")
outfile = pysam.AlignmentFile("XT_.bam", "wb", template=samfile)

for read in samfile:

    if read.reference_name is not None:
        read.tags += [('XT',read.reference_name)]
    
    else:
        pass

    outfile.write(read)

samfile.close()
outfile.close()

samtools index XT.bam

Then check for number of artefacts between mouse and human chimeric reads

In [3]:
bamfile = pysam.AlignmentFile("final_sorted.bam", "rb")
split = pysam.AlignmentFile("final_SA.bam", "wb", template=bamfile)

for line in bamfile:
    if line.has_tag("SA"):
        split.write(line)
    else:
        pass

bamfile.close()
split.close()

In [34]:
bamfile = pysam.AlignmentFile("final_SA.bam", "rb")

n = 0
fusions = 0
for line in bamfile:
    n += 1
    tag = line.get_tag('SA').split(",")[0]
    contig = line.reference_name
    if contig[:4] == "ENST" and tag[:4] == "ENST":
        pass
    elif contig[:4] == "ENST" and tag[:4] == "ENSM":
        fusions += 1
        print(tag, contig)
    elif contig[:4] == "ENSM" and tag[:4] == "ENST":
        fusions += 1
        print(tag, contig)
    elif contig[:4] == "ENSM" and tag[:4] == "ENSM":
        pass

print("total number of reads:{}".format(n))
print("total number of reads with a false chimera:{}".format(fusions))
print("percent of reads that are fusions: {}".format(fusions/n*100))

ENSMUST00000057792.8 ENST00000445125.2
ENSMUST00000057792.8 ENST00000445125.2
total number of reads:403
total number of reads with a false chimera:2
percent of reads that are fusions: 0.49627791563275436
